In [132]:
import re
import pandas as pd
import numpy as np
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [32]:
#you need to manually run your search and add your URL resulting from the search, 
#checking name, seq, Pubmed ID and hemolytic activity.
text = """ web input"""
browser = Browser()
response = browser.open("https://webs.iiitd.edu.in/raghava/hemolytik/submitkey_target_search.php?ran=6203")
content = response.read()
#seperating the data into the correct labels
split = str(content, 'utf-8').split(sep='</td><td')
clean_split = split[7:]
seq =[]
toxic = []
PMID = []
for i in range(int((len(clean_split)/4))):
    seq_pos = ((2) + 4*i)
    toxic_pos = ((3)+ 4*i)
    PMID_pos = ((0) + 4*i)
    seq.append((clean_split[seq_pos]).upper())
    toxic.append(clean_split[toxic_pos])
    PMID.append(clean_split[PMID_pos])

In [152]:
seq =[]
toxic = []
for f in range(4):
    test_list = test_list_list[f]
    for i in range(int((len(test_list)/12))):
        seq_pos = ((2) + 12*i)
        toxic_pos = ((10)+ 12*i)
        seq.append((test_list[seq_pos]).upper())
        toxic.append(test_list[toxic_pos])

In [164]:
df_hemolytik = pd.DataFrame([seq, toxic], index = ['seq', 'toxic'])
df_hemolytik = df_hemolytik.transpose()
valid = []
valid1 =[]
valid2 =[]
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.seq[n].isalpha())== False):
        valid.append(n)
    if ("X") in df_hemolytik.seq[n]:
        valid1.append(n)
    if ("O") in df_hemolytik.seq[n]:
        valid2.append(n)
    if ("Z") in df_hemolytik.seq[n]:
        valid2.append(n)
    
valid = valid + list(set(valid1) - set(valid)) 
valid = valid + list(set(valid2) - set(valid)) 
df_hemolytik = df_hemolytik.drop(valid)
df_hemolytik = df_hemolytik.reset_index(drop=True)

In [165]:
#automatic read out
df_hemolytik['toxic2'] = np.nan
df_hemolytik['toxic3'] = "other"
df_hemolytik['toxic4'] = "other"
df_hemolytik['toxic5'] = "other"
for n in range(len(df_hemolytik)):
    if "-" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic']= re.sub('-[^>]+[%μµnm ]', '', df_hemolytik.toxic[n])
    if "±" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic']= re.sub('±[^>]+[%μµnm ]', '', df_hemolytik.toxic[n])
    if "MHC" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic'] = df_hemolytik.toxic[n].replace('MHC', '15% hem')
    if "M" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "M"
    if "μM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μM"
    if "µM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μM"
    if "µg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "μg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "μg/mL" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "µg/mL" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"        
    if "nM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "nM"
    if "mg/L" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mg/L"
    if "mg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mg/ml"        
    if "mM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mM"
    df_hemolytik.toxic2[n] = re.findall(r"[-+]?\d*\.\d+|\d+", str(df_hemolytik.toxic[n]))
    if (len(df_hemolytik.toxic2[n])==2):
        df_hemolytik.loc[n, 'toxic4'] = (df_hemolytik.toxic2[n])[0]
        df_hemolytik.loc[n, 'toxic5'] = (df_hemolytik.toxic2[n])[1]
df_hemolytik

,seq,toxic,toxic2,toxic3,toxic4,toxic5
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,2% hemolysis upto 100μg/ml,"[2, 100]",μg/ml,2,100
1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,9% hemolysis upto 200μg/ml,"[9, 200]",μg/ml,9,200
2,AAAKAALNAVLVGANA,LC<sub>50</sub> =40μM,"[50, 40]",μM,50,40
3,AAGLAMLFLGILSAAGSTMGARA,~50% hemolysis at 40µM,"[50, 40]",μM,50,40
4,AAGLAMLFLGILSAAGSTMGARA,55M,[55],M,other,other
...,...,...,...,...,...,...
2534,IWRIFR,15% hem &gt;256μM,"[15, 256]",μM,15,256
2535,IWRIFRRIF,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4
2536,FGVLAKVAAHVVPAIAEHF,0% hemolysis upto 100µM,"[0, 100]",μM,0,100
2537,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6


In [166]:
unvalid1= []
unvalid2= []
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.toxic3[n]== 'other')):
        unvalid1.append(n)
    if ((df_hemolytik.toxic4[n]== 'other')):
        unvalid2.append(n)
unvalid = unvalid1 + list(set(unvalid2) - set(unvalid1))
df_hemolytik = df_hemolytik.drop(unvalid)
df_hemolytik.reset_index(drop=True)

,seq,toxic,toxic2,toxic3,toxic4,toxic5
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,2% hemolysis upto 100μg/ml,"[2, 100]",μg/ml,2,100
1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,9% hemolysis upto 200μg/ml,"[9, 200]",μg/ml,9,200
2,AAAKAALNAVLVGANA,LC<sub>50</sub> =40μM,"[50, 40]",μM,50,40
3,AAGLAMLFLGILSAAGSTMGARA,~50% hemolysis at 40µM,"[50, 40]",μM,50,40
4,AARRAARRAARR,HC<sub>50</sub> &gt; 500mg/L,"[50, 500]",mg/L,50,500
...,...,...,...,...,...,...
2026,IWRIFR,15% hem &gt;256μM,"[15, 256]",μM,15,256
2027,IWRIFRRIF,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4
2028,FGVLAKVAAHVVPAIAEHF,0% hemolysis upto 100µM,"[0, 100]",μM,0,100
2029,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6


In [167]:
df_hemolytik = df_hemolytik.reset_index(drop=True)
#types of units that need conversion "μg/ml", "nM", "M", "mM", "mg/L", "mg/ml"
glob = GlobalDescriptor(df_hemolytik.seq.tolist())
glob.calculate_MW(amide=True)
df_hemolytik['MW'] = glob.descriptor
#see what concentrations are in µg/ml instead of µM
mask0 = df_hemolytik["toxic3"] == 'µg/ml'
mask1 = df_hemolytik["toxic3"] == 'mg/L'
mask2 = df_hemolytik["toxic3"] == 'mg/ml'
mask3 = df_hemolytik["toxic3"] == 'μM'
mask4 = df_hemolytik["toxic3"] == 'nM'
mask5 = df_hemolytik["toxic3"] == 'mM'
mask6 = df_hemolytik["toxic3"] == 'M'
#calculate the µM
df_hemolytik["CONCENTRATION_µM"] = 0
df_hemolytik.loc[mask0, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask0])*1000) / df_hemolytik[mask0]['MW']
df_hemolytik.loc[mask1, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask1]).astype(float)*1000) / df_hemolytik[mask1]['MW']
df_hemolytik.loc[mask2, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask2]).astype(float)*1000000) / df_hemolytik[mask2]['MW']
df_hemolytik.loc[mask4, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask4]).astype(float)/1000)
df_hemolytik.loc[mask5, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask5]).astype(float)*1000)
df_hemolytik.loc[mask6, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask6]).astype(float)*1000000)
#fill in values already in µM
df_hemolytik.loc[mask3, "CONCENTRATION_µM"] = df_hemolytik["toxic5"][mask3].astype(float)
df_hemolytik

,seq,toxic,toxic2,toxic3,toxic4,toxic5,MW,CONCENTRATION_µM
0,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,2% hemolysis upto 100μg/ml,"[2, 100]",μg/ml,2,100,3176.71,0
1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,9% hemolysis upto 200μg/ml,"[9, 200]",μg/ml,9,200,3176.71,0
2,AAAKAALNAVLVGANA,LC<sub>50</sub> =40μM,"[50, 40]",μM,50,40,1423.67,40.00
3,AAGLAMLFLGILSAAGSTMGARA,~50% hemolysis at 40µM,"[50, 40]",μM,50,40,2149.59,40.00
4,AARRAARRAARR,HC<sub>50</sub> &gt; 500mg/L,"[50, 500]",mg/L,50,500,1380.61,362.16
...,...,...,...,...,...,...,...,...
2026,IWRIFR,15% hem &gt;256μM,"[15, 256]",μM,15,256,889.10,256.00
2027,IWRIFRRIF,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4,1305.62,101.40
2028,FGVLAKVAAHVVPAIAEHF,0% hemolysis upto 100µM,"[0, 100]",μM,0,100,1975.35,100.00
2029,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6,3498.40,12.60


In [120]:
#select wether toxicity is positive or negative
#>=5% cell death with <=10uM peptide
#>=10% cell death with <=20uM peptide
#>=15% cell death with <=50uM peptide
#>=10% cell death with <=100uM peptide
#>=30% cell death with <=200uM peptide
#>=50% cell death with <=300uM peptide
#neg if not or 
#<=5% cell death with >=50uM
df_hemolytik["toxic6"] = "~"
for v in range(len(df_hemolytik)):
    if float(df_hemolytik["toxic4"][v]) <= 5:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 50:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 5:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 10:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 10:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 20:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 15:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 50:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 20:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 100:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0
    if float(df_hemolytik["toxic4"][v]) >= 30:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 200:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0            
    if float(df_hemolytik["toxic4"][v]) >= 50:
        if float(df_hemolytik["CONCENTRATION_µM"][v]) <= 300:
            df_hemolytik.loc[v, "toxic6"] = 1
        else:
            df_hemolytik.loc[v, "toxic6"] = 0 

KeyError: '[ True False False ... False False False] not found in axis'

In [168]:
#HAPPENN formating of positive/negative
#happenn formating
df_hemolytik["toxic6"] = "~"
for v in range(len(df_hemolytik)):
    if float(df_hemolytik["toxic4"][v]) <= 5:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 30:
            df_hemolytik.loc[v, "toxic6"] = 0        
    if 5 <= float(df_hemolytik["toxic4"][v]) <= 10:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 60:
            df_hemolytik.loc[v, "toxic6"] = 0   
    if 10 <= float(df_hemolytik["toxic4"][v]) <= 15:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 90:
            df_hemolytik.loc[v, "toxic6"] = 0
    if 15 <= float(df_hemolytik["toxic4"][v]) <= 20:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 120:
            df_hemolytik.loc[v, "toxic6"] = 0
    if 20 <= float(df_hemolytik["toxic4"][v]) <= 25:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 180:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 25 <= float(df_hemolytik["toxic4"][v]) <= 30:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 180:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 30 <= float(df_hemolytik["toxic4"][v]) <= 35:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 210:
            df_hemolytik.loc[v, "toxic6"] = 0         
    if 35 <= float(df_hemolytik["toxic4"][v]) <= 40:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 240:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 40 <= float(df_hemolytik["toxic4"][v]) <= 45:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 270:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if float(df_hemolytik["toxic4"][v]) >= 50:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 300:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 55:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 330:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 60:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 360:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 65:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 390:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 70:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 420:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 75:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 450:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 80:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 480:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 85:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 510:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 90:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 540:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 95:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 570:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 100:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 600:
            df_hemolytik.loc[v, "toxic6"] = 1 

In [169]:
unvalid1= []
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.toxic6[n]== '~')):
        unvalid1.append(n)
df_hemolytik = df_hemolytik.drop(unvalid1)
df_hemolytik = df_hemolytik.reset_index(drop=True)

In [171]:
peptides = df_hemolytik["seq"].unique()
final_data = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(df_hemolytik)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if df_hemolytik["seq"][a] == peptides[n]:
            toxic.append(df_hemolytik["toxic6"][a])
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                final_data.loc[n, "toxic"] = "~"
            if toxic_count <= len(toxic)/2:
                final_data.loc[n, "toxic"] = 0
            if toxic_count >= len(toxic)/2:
                final_data.loc[n, "toxic"] = 1

In [174]:

final_data.to_csv(Root + "/Programing/Data/AMP_focussed_data/hemolytic_data.csv",index = False, header=True)

In [172]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from lazypredict.Supervised import LazyClassifier
from DNN_features import *
test2 = AAC_df(final_data.seq)
train_X, val_X, train_y, val_y = train_test_split(test2, final_data.toxic, random_state = 445)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
#models_train,predictions_train = clf.fit(train_X, train_X, train_y, train_y)
models,predictions = clf.fit(train_X, val_X, train_y, val_y)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 19.56it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,matthews_corrcoef,Time Taken
Model,,,,,,
ExtraTreesClassifier,0.90,0.89,0.89,0.90,0.78,0.17
LabelPropagation,0.88,0.88,0.88,0.88,0.76,0.04
LabelSpreading,0.88,0.88,0.88,0.88,0.76,0.04
XGBClassifier,0.87,0.87,0.87,0.87,0.73,0.08
LGBMClassifier,0.86,0.85,0.85,0.86,0.71,0.07
NuSVC,0.84,0.80,0.80,0.83,0.66,0.03
ExtraTreeClassifier,0.80,0.80,0.80,0.80,0.59,0.01
BaggingClassifier,0.80,0.80,0.80,0.80,0.59,0.06
RandomForestClassifier,0.83,0.80,0.80,0.82,0.63,0.24


In [173]:
final_data.toxic.value_counts()

1.00    446
0.00    239
Name: toxic, dtype: int64